In [2]:
import ROOT
import uproot
import numpy as np
import matplotlib.pyplot as plt

In [3]:
with open("TUnfoldHistoFileList_Nominal_emu.txt", "r") as ipfile :
    allfiles = ipfile.readlines()

In [4]:
BASE_DIR = "/depot/cms/top/bakshi3/TopSpinCorr_Run2_generalized_ND/CMSSW_10_6_30/src/TopAnalysis/Configuration/analysis/diLeptonic/"

In [5]:
def get_lumi_from_samplename(samplename) :
    # 2016
    if   "2016ULpreVFP"  in samplename : lumi = 19668.0
    elif "2016ULpostVFP" in samplename : lumi = 16778.0
    elif "2016UL" in samplename : lumi = 36310.0
    
    # 2017
    elif "2017UL" in samplename : lumi = 41480.0
    
    # 2018
    elif "2018UL" in samplename : lumi = 59830.0
    
    # full Run2
    elif "fullRun2UL" in samplename : lumi = 137650.0
    
    return lumi

In [6]:
def get_dySF_from_samplename(samplename,channel) :
    dySF = 1.0 

    if  "2018" in samplename:
        if    channel == "ee"       : dySF = 1.12717
        elif  channel == "emu"      : dySF = 1.10941
        elif  channel == "mumu"     : dySF = 1.09194
        elif  channel == "combined" : dySF = 1.1027
    
    elif "2017" in samplename :
        if    channel == "ee"       : dySF = 1.17378
        elif  channel == "emu"      : dySF = 1.16718
        elif  channel == "mumu"     : dySF = 1.16061
        elif  channel == "combined" : dySF = 1.16462

    elif "2016postVFP" in samplename :
        if    channel == "ee"       : dySF = 1.19939
        elif  channel == "emu"      : dySF = 1.1901
        elif  channel == "mumu"     : dySF = 1.18087
        elif  channel == "combined" : dySF = 1.18599

    elif "2016preVFP" in samplename :
        if    channel == "ee"       : dySF = 1.1809
        elif  channel == "emu"      : dySF = 1.14192
        elif  channel == "mumu"     : dySF = 1.10422
        elif  channel == "combined" : dySF = 1.12537

    elif "2016" in samplename and not "VFP" in samplename: 
        if    channel == "ee"       : dySF = 1.18945
        elif  channel == "emu"      : dySF = 1.16376
        elif  channel == "mumu"     : dySF = 1.13863
        elif  channel == "combined" : dySF = 1.15266
            
    elif "fullRun2UL" in samplename :
        if    channel == "ee"       : dySF = 1.15707
        elif  channel == "emu"      : dySF = 1.14002
        elif  channel == "mumu"     : dySF = 1.12321
        elif  channel == "combined" : dySF = 1.13327
    
    return dySF 

In [7]:
def get_cross_section_from_samplename(samplename, channel) :
    topxsec = 830.91
    
    # w/o tau signal files
    if   "ee_ttbarsignalplustau_fromDilepton"   in samplename : xsection = topxsec * 0.10706 * 0.964261576
    elif "emu_ttbarsignalplustau_fromDilepton"  in samplename : xsection = topxsec * 0.10706 * 0.957058875
    elif "mumu_ttbarsignalplustau_fromDilepton" in samplename : xsection = topxsec * 0.10706 * 0.949909976
    
    # w tau signal files
    elif "ee_ttbarsignalviatau_fromDilepton"   in samplename : xsection = topxsec * 0.10706 * 1.029827957
    elif "emu_ttbarsignalviatau_fromDilepton"  in samplename : xsection = topxsec * 0.10706 * 1.026209047
    elif "mumu_ttbarsignalviatau_fromDilepton" in samplename : xsection = topxsec * 0.10706 * 1.022670477
    
    # backgrounds
    
    # ttbar backgrounds
    elif "bg_fromDilepton" in samplename : xsection = topxsec * 0.10706
    elif "fromLjets"       in samplename : xsection = topxsec * 0.44113
    elif "fromHadronic"    in samplename : xsection = topxsec * 0.45441
    
    # elif ("ttbar"  in samplename) and not("ttbarW" in samplename) and not("ttbarW" in samplename) : xsection = topxsec 
    
    # Single top
    elif ("single" in samplename)    and ("tw" in samplename) : xsection = 35.85 * 0.54559
    elif ("single" in samplename)    and ("_s" in samplename) : xsection = 10.32
    elif ("singletop" in samplename) and ("_t" in samplename) : xsection = 136.02

    # Single antitop
    elif ("singleantitop" in samplename) and ("_t" in samplename) : xsection = 80.95
    
    # VV
    elif "ww" in samplename : xsection = 118.7
    elif "wz" in samplename : xsection = 47.13
    elif "zz" in samplename : xsection = 16.523
    
    # Drell-Yan
    elif "1050" in samplename               : xsection = 22635.1 * get_dySF_from_samplename(samplename, channel)
    elif "50inf_amcatnlofxfx" in samplename : xsection = 0.5 * 3.*2075.14 * get_dySF_from_samplename(samplename, channel)
    elif "50inf_madgraphmlm"  in samplename : xsection = 0.5 * 3.*2075.14 * get_dySF_from_samplename(samplename, channel)
    
    # Smaller backgrounds
    elif "wtolnu"          in samplename : xsection = 61526.7
    elif "ttbarWjetstolnu" in samplename : xsection = 0.2043
    elif "ttbarWjetstoqq"  in samplename : xsection = 0.4062
    elif "ttbarZtollnunu"  in samplename : xsection = 0.2529
    elif "ttbarZtoqq"      in samplename : xsection = 0.5297
    else : xsection = topxsec
    
    return xsection

In [8]:
# get_cross_section_from_samplename(file, "emu")

In [9]:
def get_lumi_weight(root_fileptr, lumi, xsection) :
    nevents = root_fileptr.Get('hNrOfEvts').GetBinContent(1)
    lumi_wt = (lumi * xsection) / nevents
    return lumi_wt

In [12]:
# Over all files
for file in allfiles :
    file = file.replace('\n','')
    # file = file.replace('UnfoldingHistos_2017', 'UnfoldingHistos_2017')
    
    # Skip data, it can just be hadded
    if "run" in file : continue
    
    print('Working with file ::' + file)
    
    uproot_fileptr = uproot.open(BASE_DIR + file)
    
    infile  = ROOT.TFile.Open(BASE_DIR + file, "READ")
    outname = file.replace('UnfoldingHistos_2017/Nominal/emu/', '')
    outname = outname.replace('.root', '_lumi_scaled.root')
    outfile = ROOT.TFile.Open(outname, "RECREATE")
        
    print('Finished setting up ROOT pointers')
    
    outfile.cd()
    
    # Over all histograms
    for k in uproot_fileptr.keys() :
        
        input_hist    = infile.Get(k)
        lumi          = get_lumi_from_samplename(file)
        xsection      = get_cross_section_from_samplename(file, "emu")
        scale_factor  = get_lumi_weight(infile, lumi, xsection)
        input_hist.Scale(scale_factor)
        input_hist.Write()
        
    outfile.Write()
    outfile.Close()
    infile.Close()

Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_wwtoall_2017UL.root
Finished setting up ROOT pointers
Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_wztoall_2017UL.root
Finished setting up ROOT pointers
Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_zztoall_2017UL.root
Finished setting up ROOT pointers
Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_ttbarWjetstolnu_2017UL.root
Finished setting up ROOT pointers
Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_ttbarWjetstoqq_2017UL.root
Finished setting up ROOT pointers
Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_ttbarZtollnunu_2017UL.root
Finished setting up ROOT pointers
Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_ttbarZtoqq_2017UL.root
Finished setting up ROOT pointers
Working with file ::UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_singleantitop_tw_2017UL.root


IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
Incr

In [90]:
outname

'UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_wtolnu_2017UL_lumi_scaled.root'

In [80]:
file.replace('.root', '_lumi_scaled.root')

'UnfoldingHistos_2017_old_minitrees/Nominal/emu/histosTUnfold_emu_ttbarsignalviatau_fromDilepton_2017UL_lumi_scaled.root'

In [82]:
outname = file.strip('UnfoldingHistos_2017_old_minitrees/Nominal/emu/')
# outname = outname.replace('.root', '_lumi_scaled.root')
outname

'histosTUnfold_emu_ttbarsignalviatau_fromDilepton_2017UL.'

In [85]:
file.replace('UnfoldingHistos_2017_old_minitrees/Nominal/emu/', '')

'histosTUnfold_emu_ttbarsignalviatau_fromDilepton_2017UL.root'

In [89]:
file

'UnfoldingHistos_2017/Nominal/emu/histosTUnfold_emu_wtolnu_2017UL.root'